In [ ]:
import pandas as pd
from pathlib import Path
import json
import gzip
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
#%matplotlib widget

In [ ]:
DATA_FOLDER = "data"
CHARGE_FOLDER = "charge"
STATIONS_FOLDER = "stations"

In [ ]:
df_charge = pd.read_parquet(Path("data/charge/"), engine="pyarrow")
df_charge = df_charge.drop('DATE', axis=1) # only used for partitioning
df_charge.sort_values(by=["TIME", "STATION_ID"], inplace=True)
df_charge.head(5)

In [ ]:
# Load the newest file in the folder ()

stations_path = Path(DATA_FOLDER) / STATIONS_FOLDER 
static_path = Path(DATA_FOLDER) / STATIONS_FOLDER
files = glob.glob(str(static_path / "stations_*.json.gz"))
latest_file = max(files, key=lambda x: Path(x).stat().st_mtime)

with gzip.open(latest_file, "rt", encoding="utf-8") as file:
	stations = json.load(file)["EVSEData"]

In [ ]:
# Series of operator IDs to names
ser_operators = pd.Series({operator['OperatorID']: operator['OperatorName'] for operator in stations})
ser_operators.head(5)

In [ ]:
# A row for each station
rows = []
for operator in stations:
    for station in operator['EVSEDataRecord']:
        row = station
        row['Operator'] = operator['OperatorID']
        rows.append(row)
df_stations = pd.DataFrame(rows)

df_stations.head(5) # 17512 stations

## Visualize a particular station

In [ ]:
#df_stat = df_charge[df_charge["STATION_ID"] == "+41*029*169*1"]
#df_stat = pd.DataFrame(df_charge[df_charge["STATION_ID"] == "+41*029*1785*2"])
df_stat = pd.DataFrame(df_charge[df_charge["STATION_ID"] == "CH*AVI*E10048"])
df_stat.sort_values("TIME", inplace=True)
fig = plt.figure(figsize=(15, 5))
plt.step(df_stat["TIME"], df_stat["STATUS"], where='post');

## Possible pre-processing step

### Keep in long format and resample

In [ ]:
df_charge_resampled = df_charge.set_index('TIME').groupby('STATION_ID').resample('30s', include_groups=False).ffill().reset_index()
df_charge_resampled.tail(5)

In [ ]:
# Variant: resample at common time index of all events (best alignment with data collection times)

time_index = (
    df_charge["TIME"]
    .sort_values()
    .unique()
)

df_charge_resampled = (
    df_charge.set_index("TIME")
      .groupby("STATION_ID", group_keys=False)
      .apply(lambda g: g.reindex(time_index).ffill(), include_groups=True)
      .reset_index()
)

df_charge_resampled.tail(5)

### Pivot to wide format

In [ ]:
# This automatically set all stations to the same time basis including all measured time steps
df_charge_pvt = df_charge.pivot(index='TIME', columns='STATION_ID', values='STATUS')
df_charge_pvt.ffill(inplace=True) # This is the performance killer somehow!
df_charge_pvt.head(5)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot((df_charge_pvt == 'Occupied').sum(axis=1), label='Occupied')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m %H:%M'))
plt.grid()
plt.legend();

## Work in the original long format without resampling

This is way faster but requires custom logic for occupancy calculation

In [ ]:
is_occupied = df_charge["STATUS"].eq("Occupied").astype(int)
df_charge['delta'] = (
    is_occupied
    .groupby(df_charge["STATION_ID"])
    .diff()
    .fillna(is_occupied)
)

occupied_count = (
    df_charge.groupby("TIME")['delta']
      .sum()
      .cumsum()
)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(occupied_count, label='Occupied')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m %H:%M'))
plt.grid()
plt.legend();

In [ ]:
for operator_id, operator_name in ser_operators.items():
    points = df_stations.query('Operator == @operator_id')['EvseID']

    occupied_count = (
        df_charge[df_charge['STATION_ID'].isin(points)]
        .groupby("TIME")['delta']
        .sum()
        .cumsum()
    )
    plt.figure(figsize=(15, 5))
    plt.plot(occupied_count)
    plt.title(operator_name)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m %H:%M'))
    plt.grid()

In [ ]:
# Swisscharge status always unknown?
points_sc = df_stations.query('Operator == "CH*SWISSCHARGE"')['EvseID']
df_charge_sc = df_charge[df_charge['STATION_ID'].isin(points_sc)]
df_charge_sc['STATUS'].value_counts()